# GSModutils demo
## shikimate production in *E. coli*

PROBLEMS/TODO LIST:

TEST FILES NEED TO BE MANUALLY AMENDED BEFORE THEY WORK (WITH ADDITIONAL DICT LAYER)

DAHMS PATHWAY TEST DOESN'T DETECT PATHWAY ACTIVITY, CONTRADICTING FBA SOLTION COMPUTED BELOW

In [1]:
# (automatic reload for use with development of gsmodutils)
%load_ext autoreload
%autoreload

In [2]:
from gsmodutils import GSMProject
import cobra

In [ ]:
# # Initialise the GSModutils project
# # Note, running more than once will throw an error.
# # Projects can't be created in the folder more than once.

# model = cobra.io.load_json_model('./iJO166.json')
# email_address = 'your_email@address.com'  # provide an email address
# author_name = ''  # author's name

# project = GSMProject.create_project(
#     models=[model],
#     description='Shikimate production in E. coli',
#     author=author_name,
#     author_email=email_address,
#     project_path='.'  # use current directory
# )

In [111]:
project = GSMProject('.')  # load project from current directory

In [112]:
m = project.model  # load model

## First, define conditions under which *E. coli* strains will be grown...

In [113]:
# glucose + xylose + M9 minimal media conditions (microaerobic)
m.exchanges.get_by_id('EX_xyl__D_e').bounds = (-10,1000)  # allow xylose uptake
m.reactions.get_by_id('EX_tyr__L_e').bounds = (-10,1000)  # allow tyrosine uptake
m.reactions.get_by_id('EX_phe__L_e').bounds = (-10,1000)  # allow phenylalanine uptake
m.reactions.get_by_id('EX_o2_e').bounds = (-0.1,0)  # microaerobic conditions
m.reactions.get_by_id('EX_co2_e').bounds = (0,1000)  # co2 product only

project.save_conditions(model=m, conditions_id='glc+xyl_M9_microaero')

In [114]:
# glucose + xylose + M9 minimal media conditions (aerobic)
m.reactions.get_by_id('EX_o2_e').bounds = (-1000,0)  # aerobic conditions

project.save_conditions(model=m, conditions_id='glc+xyl_M9_aero')

In [115]:
# glucose + M9 minimal media conditions (aerobic)
m.exchanges.get_by_id('EX_xyl__D_e').bounds = (0,1000)  # block xylose uptake

project.save_conditions(model=m, conditions_id='glc_M9_aero')

In [116]:
# xylose + M9 minimal media conditions (aerobic)
m.exchanges.get_by_id('EX_xyl__D_e').bounds = (-10,1000)  # allow xylose
m.exchanges.get_by_id('EX_glc__D_e').bounds = (0,1000)  # block glucose uptake

project.save_conditions(model=m, conditions_id='xyl_M9_aero')

In [117]:
# create separate condition which includes malate
m.exchanges.get_by_id('EX_mal__L_e').bounds = (-15,1000)  # allow malate uptake

project.save_conditions(model=m, conditions_id='glc+xyl+mal_M9_microaero')

To check whether these conditions support growth of the default model, <br>
open a terminal, cd into the project directory and try running 

``` $ gsmodutils test```

## Create the 'CTF5' mutant strain of *E. coli*...

In [118]:
# create E. coli CTF5 strain design
# The CTF5 strain has disruptions in pts transport system and an amino-acid biosynthesis pathway (phenylalanine and tyrosine)
aa_pathway_genes = list(m.reactions.PPNDH.genes)+list(m.reactions.PPND.genes)  # amino acid pathway genes (phe/tyr)
pts_system_genes = list(m.reactions.GLCptspp.genes)  # genes encoding pts transport system genes

gene_ko_list = aa_pathway_genes + pts_system_genes + list(m.reactions.PYK.genes)

cobra.manipulation.delete_model_genes(m,gene_ko_list)

# save ctf5 design
project.save_design(m,
                    'CTF5',
                    'CTF5 base strain',
                    conditions='glc+xyl_M9_microaero',
                    base_model='iJO1366.json'
                   )

# save ctf5 strain with malate in the 'medium'
project.save_design(m,
                    'CTF5mal',
                    'CTF5 base strain with malate',
                    conditions='glc+xyl+mal_M9_microaero',
                    base_model='iJO1366.json'
                   )

metabolites,0
reactions,325
genes,0
description,
parent,None
id,CTF5mal
name,CTF5 base strain with malate


In [119]:
# CAN A GSMODUTILS TEST BE WRITTEN FOR THIS BIT?
# E.G. FAILS IF GROWTH RATE BELOW 0.05...
ctf5 = project.load_design('CTF5')
sol = ctf5.optimize()
print('CTF5 gorwth:',sol.objective_value)  # very slow growth ~ 0.019
ctf5 = project.load_design('CTF5mal')
sol = ctf5.optimize()
print('CTF5 growth with malate:',sol.objective_value)  # ~ growth restored when malate uptake allowed

CTF5 gorwth: 0.019464018902300633
CTF5 growth with malate: 1.5616176489893787


## Create 'G1' mutant...

In [120]:
# create G1 mutant
eda_genes = list(ctf5.reactions.EDA.genes)
pps_genes = list(ctf5.reactions.PPS.genes)
ppc_genes = list(ctf5.reactions.PPC.genes)
ppck_genes = list(ctf5.reactions.PPCK.genes)

knockout_targets = eda_genes + pps_genes + ppc_genes + ppck_genes
cobra.manipulation.delete_model_genes(ctf5,knockout_targets)
project.save_design(ctf5,
                    'G1',
                    'ctf5 with additional KOs',
                    conditions='glc+xyl_M9_microaero',
                    base_model='iJO1366.json'
                   )

metabolites,0
reactions,35
genes,0
description,
parent,None
id,G1
name,ctf5 with additional KOs


Now repeat the test run from the command line, what do the results say about the G1 strain?

## Create 'G1x' mutant by adding the 'Dahms' pathway...

In [121]:
# make a function to easily add new reactions to a cobra model
def add_new_reaction(m,reaction_id,stoich_dict,compartment='c'):
    reaction_dict = {}
    model_metabolite_ids = [
        x.id for x in m.metabolites
    ]
    for metabolite_id in stoich_dict.keys():
        if metabolite_id[-2:] == '_e':
            compartment = 'e'
        if metabolite_id in model_metabolite_ids:
            metabolite = m.metabolites.get_by_id(metabolite_id)
            reaction_dict[metabolite] = stoich_dict[metabolite_id]
        else:
            metabolite = cobra.Metabolite(metabolite_id,compartment=compartment)
            reaction_dict[metabolite] = stoich_dict[metabolite_id]
    reaction = cobra.Reaction(reaction_id)
    reaction.add_metabolites(reaction_dict)
    m.add_reactions([reaction])

In [132]:
g1 = project.load_design('G1')  # load G1 design

In [133]:
# define xylose dehydrogenase (xdh)
xdh_stoich_dict = {
    # substrates
    'xyl__D_c': -1,
    'nad_c': -1,
    # products
    'xylonolactone_c': 1,
    'nadh_c': 1,
    'h_c': 1
}
add_new_reaction(g1,'xdh',xdh_stoich_dict)
print(g1.reactions.xdh)

xdh: nad_c + xyl__D_c --> h_c + nadh_c + xylonolactone_c


In [134]:
# define xylonolactonase (xylC)
xylC_stoich_dict = {
    # substrates
    'xylonolactone_c': -1,
    'h2o_c': -1,
    # products
    'xylonate_c': 1,
    'h_c': 1
}
add_new_reaction(g1,'xylC',xylC_stoich_dict)
print(g1.reactions.xylC)

xylC: h2o_c + xylonolactone_c --> h_c + xylonate_c


In [135]:
# define xylonate dehydratase
stoich_dict = {
    'xylonate_c': -1,
    'h2o_c': 1,
    'dehydrodeoxylonate_c': 1
}
add_new_reaction(g1,'yjhG',stoich_dict)
print(g1.reactions.yjhG)

yjhG: xylonate_c --> dehydrodeoxylonate_c + h2o_c


In [136]:
# define aldolase
stoich_dict = {
    'dehydrodeoxylonate_c': -1,
    'pyr_c': 1,
    'gcald_c': 1
}
add_new_reaction(g1,'yjhH',stoich_dict)
print(g1.reactions.yjhH)

yjhH: dehydrodeoxylonate_c --> gcald_c + pyr_c


In [137]:
# define a list of the Dahms pathway reaction IDs
dahms_pathway_reactions = [
    'xdh',
    'xylC',
    'yjhG',
    'yjhH'
]

In [140]:
# make shikimate transport reversible, rather than 'uptake-only'
g1.reactions.SKMt2pp.bounds = (-1000,1000)
g1.reactions.get_by_id('EX_o2_e').bounds = (-1000,0)  # aerobic conditions

In [141]:
# create G1x mutant design
project.save_design(g1,
                    'G1x',
                    'G1 with Dahms pathway',
                    conditions='glc+xyl_M9_aero',
                    base_model='iJO1366.json'
                   )

metabolites,3
reactions,40
genes,0
description,
parent,None
id,G1x
name,G1 with Dahms pathway


In [142]:
g1x = project.load_design('G1x')

In [146]:
## BUG? test_Dahms seems unable to reproduce this result
# compute an FBA solution to see if g1x grows, and check if Dahms pathway is active
sol = g1x.optimize()
v = sol.fluxes
print('growth:',v['BIOMASS_Ec_iJO1366_core_53p95M'])
for reaction in dahms_pathway_reactions:
    print(reaction,v[reaction])

    # display active transport fluxes
for ex in g1x.exchanges:
    if abs(v[ex.id]) > 1e-10:
        print(ex,v[ex.id])

growth: 1.7718192826997718
xdh 5.243447568811833
xylC 5.243447568811833
yjhG 5.243447568811833
yjhH 5.243447568811833
EX_cobalt2_e: cobalt2_e <=>  -4.4295482067494296e-05
EX_h_e: h_e <=>  15.706908624602264
EX_h2o_e: h2o_e <=>  86.35491929591316
EX_k_e: k_e <=>  -0.3458467212480166
EX_cu2_e: cu2_e <=>  -0.0012562198714341382
EX_meoh_e: meoh_e -->  3.5436385653995436e-06
EX_mg2_e: mg2_e <=>  -0.015370532277420522
EX_mn2_e: mn2_e <=>  -0.0012243271243455422
EX_mobd_e: mobd_e <=>  -0.00022856468746827055
EX_nh4_e: nh4_e <=>  -18.564130225329794
EX_ca2_e: ca2_e <=>  -0.009222319366452313
EX_ni2_e: ni2_e <=>  -0.0005722976283120263
EX_cl_e: cl_e <=>  -0.009222319366452313
EX_pi_e: pi_e <=>  -1.7091518064899376
EX_zn2_e: zn2_e <=>  -0.0006041903754006222
EX_so4_e: so4_e <=>  -0.4468705412897089
EX_fe2_e: fe2_e <=>  -0.02845718949944103
EX_co2_e: co2_e -->  42.42648022374612
EX_glc__D_e: glc__D_e <=>  -10.0
EX_o2_e: o2_e <--  -39.096201478640296
EX_phe__L_e: phe__L_e <=>  -0.3282560994093733


## Create 'essential pathway' tests to monitor key pathways...

In [ ]:
# define a list of reaction IDs for reactions involved in the TCA cycle (aka 'Citric Acid Cycle')
tca_cycle = [
    x.id for x in g1x.reactions
    if x.subsystem == 'Citric Acid Cycle'
]
# add an essential pathway test to monitor TCA cycle reactions
project.add_essential_pathway(
    'TCA',
    tca_cycle,
    'reactions of the tca cycle',
    conditions=[
        'glc+xyl_M9_aero',
        'glc_M9_aero',
        'xyl_M9_aero'],
    designs=[
        "G1x"
    ]
)

In [145]:
# add an essential pathway test to monitor Dahms pathway reactions
project.add_essential_pathway(
    'Dahms',
    dahms_pathway_reactions,
    'reactions in Dahms pathway',
    conditions=[
        'glc+xyl_M9_aero',
        'glc_M9_aero',
        'xyl_M9_aero'],
    designs=[
        "G1x"
    ]
)

the above commands create test-files written to the test directory as test_TCA and test_Dahms.

## Test shikimate production in G1x strain...

In [148]:
g1x.exchanges.EX_skm_e.bounds = (1,1000)  # shikimate efflux fixed
# g1x.reactions.SKMt2pp.bounds = (-1000,1000)  # make shikimate transport reversible, rather than 'uptake-only'
g1x.objective = 'BIOMASS_Ec_iJO1366_core_53p95M'
sol = g1x.optimize()
print('solver status:',sol.status)
print('growth:',sol.objective_value)

# # show active exchange reactions
# v = sol.fluxes  
# for ex in g1x.exchanges:
#     if abs(v[ex.id]) > 1e-10:
#         print(ex,v[ex.id])

solver status: optimal
growth: 1.6519053443710339


In [149]:
# create some designs with shikimate production lower bound
project.save_design(g1x,
                    'G1x_skm_glcxyl',  
                    'G1 with Dahms pathway and shikimate efflux lower bound (glc+xyl)',
                    conditions='glc+xyl_M9_aero',  # glucose and xylose
                    base_model='iJO1366.json'
                   )

project.save_design(g1x,
                    'G1x_skm_glc',  # 
                    'G1 with Dahms pathway and shikimate efflux lower bound (glc)',
                    conditions='glc_M9_aero',  # glucose
                    base_model='iJO1366.json'
                   )

project.save_design(g1x,
                    'G1x_skm_xyl',
                    'G1 with Dahms pathway and shikimate efflux lower bound (xyl)',
                    conditions='xyl_M9_aero',  # xylose
                    base_model='iJO1366.json'
                   )

metabolites,3
reactions,41
genes,0
description,
parent,None
id,G1x_skm_xyl
name,G1 with Dahms pathway and shikimate efflux lower bound (xyl)


In [151]:
# designs with shikimate-maximizing objective should be tested for shikimate production under different conditions
project.add_essential_pathway(
    'shikimate_production',
    description='test if shikimate boundary is active in efflux direction (minimum 0.01 efflux)',
    reactions=[],
    reaction_fluxes={
        'EX_skm_e': (0.01, 1000)
    },
    conditions=[
        'glc_M9_aero',
        'xyl_M9_aero',
        'glc+xyl_M9_aero'
    ],
    designs=[
        'G1x_skm_xyl',
        'G1x_skm_glc',
        'G1x_skm_glcxyl'
    ],
    models=['iJO1366.json'] 
)

In [ ]:
# Had to run the following command to get gsmodutils shell commands to work:
# set PATH=%PATH%;C:\Users\Admin\AppData\Roaming\Python\Python35\Scripts

In [ ]:
# m.metabolites.get_by_id('tyr__L_e').bounds = (-1000,1000)  # allow tyrosine uptake
# m.metabolites.get_by_id('trp__L_e').bounds = (-1000,1000)  # allow tryptophan uptake
# m.metabolites.get_by_id('phe__L_c').bounds = (-1000,1000)  # allow phenylalanine uptake

In [ ]:
# m.exchanges.get_by_id('EX_xyl__D_e').bounds = (-10,1000)  # allow xylose uptake
# sol = m.optimize()
# print('growth rate (glc+xyl):',sol.objective_value)
# project.save_conditions(model=m, conditions_id='glc_and_xyl_growth')

In [ ]:
# m.exchanges.get_by_id('EX_glc__D_e').bounds = (0,1000)  # prevent glucose uptake
# sol = m.optimize()
# print('growth rate (xyl):',sol.objective_value)
# project.save_conditions(m, 'xyl_growth')

In [ ]:
# m.exchanges.get_by_id('EX_glc__D_e').bounds = (-10,1000)  # allow glucose uptake
# sol = m.optimize()

# project.save_conditions(model=m, conditions_id='M9_glc_and_xyl')

In [ ]:
# # next we want to look at the pathways involved in xylose utilisation
# # authors want "...a xylose catabolic pathway that directly flows into the 
# # TCA cycle without interfering glycolysis and PPP..."
# m_xyl = project.load_conditions('xyl_growth',m)

In [ ]:
# sol = m_xyl.optimize()

In [ ]:
# v = sol.fluxes

In [ ]:
# # model reactions involved in the pentose phosphate pathway
# xyl_ppp_reactions = [
#     'XYLI1',  # xylose isomerase
#     'XYLK',  # xylose kinase
    
#     #xylulose-5p and fructose-6p to ribose-5p
#     'RPE',  # ribose phosphate isomerase
#     'RPI',  # ribose phoshpate epimerase
#     'GND',  #
#     'PGL',  #
#     'G6PDH2r',  #
#     'PGI',  #
    
#     #xylulose-5p and ribose-5p to g3p and erythrose-4p 
#     'TKT1',  # transketolase 1
#     'PFK_3',  # 
#     'FBA3',  #
#     'TPI',  #
    
#     #xylulose-5p and erythrose-4p to g3p and fructose-6p
#     'TKT2',  # transketolase 2
# ]

In [ ]:
# # display fluxes of the PPP reactions when xylose is the carbon/energy source
# for reaction in xyl_ppp_reactions:
#     print(reaction,v[reaction])

In [ ]:
# # we want to monitor the activity of this pathway, so an 'essential pathway' test is made
# # write essential pathway test, where fluxes must be within 5% of the current flux distribution

# # store reaction flux ranges in dictionary
# required_fluxes={}
# for reaction in xyl_ppp_reactions:
#     reaction_flux = v[reaction]
#     reaction_flux_margin = abs(reaction_flux*0.05)
#     required_fluxes[reaction] = [reaction_flux-reaction_flux_margin,reaction_flux+reaction_flux_margin]

# # write test file
# project.add_essential_pathway(
#     'xylose_WT_PPP',
#     description='WT pentose phosphate pathway flux on xylose',
#     reactions=[],
#     reaction_fluxes=required_fluxes,
#     conditions=['xyl_growth'],
#     models=['iJO1366.json']
# )

In [ ]:
# m_xyl_glc = project.load_conditions('glc_and_xyl_growth')

In [ ]:
# m_xyl_glc

In [ ]:
# sol = m_xyl_glc.optimize()

In [ ]:
# v = sol.fluxes

In [ ]:
# # this time get ppp flux for glc+xyl solution
# required_fluxes = {}
# for reaction in xyl_ppp_reactions:
#     if reaction in v.keys():
#         reaction_flux = v[reaction]
#         reaction_flux_margin = abs(reaction_flux*0.05)
#         required_fluxes[reaction] = [reaction_flux-reaction_flux_margin,reaction_flux+reaction_flux_margin]
#         print(reaction,reaction_flux-reaction_flux_margin,reaction_flux,reaction_flux+reaction_flux_margin)
# # write another test file
# # project.add_essential_pathway(
# #     'glucose_xylose_WT_PPP',
# #     description='WT pentose phosphate pathway flux on glucose and xylose',
# #     reactions=[],
# #     reaction_fluxes=required_fluxes,
# #     conditions=['glc_and_xyl_growth'],
# #     models=['iJO1366.json']
# # )

In [ ]:
# project.save_design(m_xyl_glc,
#                     'PK_PTS_KO',
#                     'pyruvate kinase and PTS knock-out',
#                     conditions='glc_and_xyl_growth',
#                     base_model='iJO1366.json'
#                    )